In [1]:
import pandas as pd
import requests
import json
from datetime import timedelta
import numpy as np

In [2]:
request = requests.get("https://coronavirus.data.gov.uk/downloads/json/coronavirus-cases_latest.json")
requestJson = json.loads(request.content)
ltlasDf = pd.DataFrame(requestJson['ltlas'])

In [3]:
lowerToUpperDf = pd.read_csv("data/Lower_Tier_Local_Authority_to_Upper_Tier_Local_Authority.csv")
lowerToRegionDf = pd.read_csv("data/Local_Authority_District_to_Region.csv")
population = pd.read_csv("data/population.csv")

In [4]:
print(ltlasDf.shape)

(29739, 10)


In [5]:
ltlasDf.specimenDate = pd.to_datetime(ltlasDf.specimenDate)

In [6]:
ltlasDf[ltlasDf.areaName=='Warwick'].head()

,areaCode,areaName,specimenDate,dailyLabConfirmedCases,previouslyReportedDailyCases,changeInDailyCases,totalLabConfirmedCases,previouslyReportedTotalCases,changeInTotalCases,dailyTotalLabConfirmedCasesRate
88,E07000222,Warwick,2020-07-02,0,None,None,430,None,None,301.8
1056,E07000222,Warwick,2020-06-26,1,None,None,430,None,None,301.8
1770,E07000222,Warwick,2020-06-22,1,None,None,429,None,None,301.1
2670,E07000222,Warwick,2020-06-17,2,None,None,428,None,None,300.4
3077,E07000222,Warwick,2020-06-15,2,None,None,426,None,None,299.0


In [7]:
ltlasDf = ltlasDf[['areaCode', 'areaName', 'specimenDate', 'dailyLabConfirmedCases', 'totalLabConfirmedCases']]

In [8]:
df1 = ltlasDf.drop_duplicates(subset=['areaCode','areaName'])[['areaCode','areaName']]
dtDf = pd.DataFrame(pd.date_range(ltlasDf.specimenDate.min(),ltlasDf.specimenDate.max(),freq='1 D'), columns=['specimenDate'])

df1['key'] = 0
dtDf['key'] = 0

df1 = df1.merge(dtDf, how='outer').drop(columns=['key'],axis=1)
ltlasDf = df1.merge(ltlasDf, how='left', on=['areaCode','areaName','specimenDate']).fillna(0)


In [9]:
ltlasDf = pd.merge(left=ltlasDf, 
    right=lowerToUpperDf,
    how="left", 
    left_on="areaCode",
    right_on="LTLA19CD")
ltlasDf = ltlasDf.drop(['LTLA19CD','LTLA19NM','FID'], axis=1)
ltlasDf = ltlasDf.rename(columns={"UTLA19CD" : "upperRegionCode", 'UTLA19NM' : "upperRegionName"})

ltlasDf = pd.merge(left=ltlasDf,
right=ltlasDf.groupby(['specimenDate', 'upperRegionCode'])['dailyLabConfirmedCases'].sum().reset_index(),
how="left",
left_on=['specimenDate','upperRegionCode'],
right_on=['specimenDate','upperRegionCode'],
suffixes =["","UpperRegion"])

ltlasDf = pd.merge(left=ltlasDf, 
    right=lowerToRegionDf,
    how="left", 
    left_on="areaCode",
    right_on="LAD19CD")
ltlasDf = ltlasDf.drop(['LAD19CD','LAD19NM','FID'], axis=1)
ltlasDf = ltlasDf.rename(columns={"RGN19CD" : "regionCode", 'RGN19NM' : "regionName"})

ltlasDf = pd.merge(left=ltlasDf,
right=ltlasDf.groupby(['specimenDate', 'regionCode'])['dailyLabConfirmedCases'].sum().reset_index(),
how="left",
left_on=['specimenDate','regionCode'],
right_on=['specimenDate','regionCode'],
suffixes =["","Region"])

In [10]:
ltlasDf.dailyLabConfirmedCases = ltlasDf.dailyLabConfirmedCases.fillna(0)

In [11]:
population.head()

,Code,Name,Geography,Area,Population
0,K02000001,UNITED KINGDOM,Country,"242,749","66,796,807"
1,K03000001,GREAT BRITAIN,Country,"228,956","64,903,140"
2,K04000001,ENGLAND AND WALES,Country,"151,045","59,439,840"
3,E92000001,ENGLAND,Country,"130,309","56,286,961"
4,E12000001,NORTH EAST,Region,"8,574","2,669,941"


In [12]:
population.Area = population.Area.str.replace(',','').astype('int')
population.Population = population.Population.str.replace(',','').astype('int')

In [13]:
ltlasDf = pd.merge(
left=ltlasDf,
right=population,
how="left",
left_on="areaCode",
right_on="Code")
ltlasDf = ltlasDf.drop(['Code','Name'], axis=1)

In [14]:
ltlasDf['areaMovingAverage7'] = ltlasDf.groupby('areaCode')['dailyLabConfirmedCases'].transform(lambda x: x.rolling(7, 1).mean())
ltlasDf['upperRegionMovingAverage7'] = ltlasDf.groupby('upperRegionCode')['dailyLabConfirmedCasesUpperRegion'].transform(lambda x: x.rolling(7, 1).mean())
ltlasDf['regionMovingAverage7'] = ltlasDf.groupby('regionCode')['dailyLabConfirmedCasesRegion'].transform(lambda x: x.rolling(7, 1).mean())

In [15]:
ltlasDf['rate'] = (ltlasDf['dailyLabConfirmedCases'] / ltlasDf.Population) * 100000

In [21]:
ltlasDf.groupby(['areaCode','areaName']).agg({'dailyLabConfirmedCases' : 'sum',
                                                'rate': 'sum',
                                                'Population': 'max',
                                                'Area': 'max'}).reset_index()

,areaCode,areaName,dailyLabConfirmedCases,rate,Population,Area
0,E06000001,Hartlepool,589.0,628.850240,93663.0,94.0
1,E06000002,Middlesbrough,947.0,671.726486,140980.0,54.0
2,E06000003,Redcar and Cleveland,698.0,508.931826,137150.0,245.0
3,E06000004,Stockton-on-Tees,955.0,483.916736,197348.0,205.0
4,E06000005,Darlington,601.0,562.718276,106803.0,197.0
...,...,...,...,...,...,...
310,E09000029,Sutton,997.0,483.162022,206349.0,44.0
311,E09000030,Tower Hamlets,790.0,243.267795,324745.0,20.0
312,E09000031,Waltham Forest,1007.0,363.560218,276983.0,39.0
313,E09000032,Wandsworth,1140.0,345.793003,329677.0,34.0


In [32]:
ltlasAllSumDf = ltlasDf.groupby(['areaCode','areaName']).agg({'dailyLabConfirmedCases' : 'sum',
                                                'rate': 'sum',
                                                'Population': 'max',
                                                'Area': 'max'}).reset_index()

In [33]:
# Adding Last 30 days cases and growth
tmp = ltlasDf[ltlasDf.specimenDate > (ltlasDf.specimenDate.max() - timedelta(days=30))].groupby('areaCode')['dailyLabConfirmedCases'].sum()
ltlasAllSumDf['last30dCases'] = ltlasAllSumDf['areaCode'].map(tmp)
tmp = ltlasDf[ltlasDf.specimenDate > (ltlasDf.specimenDate.max() - timedelta(days=30))].groupby('areaCode')['rate'].sum()
ltlasAllSumDf['last30dRate'] = ltlasAllSumDf['areaCode'].map(tmp)

In [34]:
# Adding Last 7 days cases and growth
tmp = ltlasDf[ltlasDf.specimenDate > (ltlasDf.specimenDate.max() - timedelta(days=7))].groupby('areaCode')['dailyLabConfirmedCases'].sum()
ltlasAllSumDf['last7dCases'] = ltlasAllSumDf['areaCode'].map(tmp)
tmp = ltlasDf[ltlasDf.specimenDate > (ltlasDf.specimenDate.max() - timedelta(days=7))].groupby('areaCode')['rate'].sum()
ltlasAllSumDf['last7dRate'] = ltlasAllSumDf['areaCode'].map(tmp)

In [35]:
tmp = ltlasDf[(ltlasDf.specimenDate >= (ltlasDf.specimenDate.max() - timedelta(days=17))) & (ltlasDf.specimenDate <= (ltlasDf.specimenDate.max() - timedelta(days=4)))].groupby('areaCode')['rate'].sum()
tmp1 = ltlasDf[(ltlasDf.specimenDate >= (ltlasDf.specimenDate.max() - timedelta(days=31))) & (ltlasDf.specimenDate <= (ltlasDf.specimenDate.max() - timedelta(days=18)))].groupby('areaCode')['rate'].sum()
ltlasAllSumDf['rFirst14'] = ltlasAllSumDf['areaCode'].map(tmp)
ltlasAllSumDf['rSecond14'] = ltlasAllSumDf['areaCode'].map(tmp1)
ltlasAllSumDf['rBasic'] = np.round(ltlasAllSumDf['rFirst14'] /  ltlasAllSumDf['rSecond14'],2)
ltlasAllSumDf['rBasic'] = ltlasAllSumDf['rBasic'].fillna(0)
ltlasAllSumDf['rBasic'] = ltlasAllSumDf['rBasic'].replace(np.inf, ltlasAllSumDf['rFirst14'])

In [40]:
ltlasAllSumDf['relativeDiff'] = ltlasAllSumDf['rFirst14'] -  ltlasAllSumDf['rSecond14']

In [42]:
ltlasAllSumDf.sort_values(by='relativeDiff', ascending=False).head(10)

,areaCode,areaName,dailyLabConfirmedCases,rate,Population,Area,last30dCases,last30dRate,last7dCases,last7dRate,rFirst14,rSecond14,rBasic,relativeDiff
15,E06000016,Leicester,3735.0,1054.417544,354224.0,73.0,1810.0,510.976106,354.0,99.936763,285.694928,200.720448,1.42,84.974479
143,E07000122,Pendle,416.0,451.624110,92112.0,169.0,83.0,90.107695,10.0,10.856349,55.367379,32.569046,1.70,22.798332
86,E07000061,Eastbourne,382.0,368.210516,103745.0,44.0,53.0,51.086799,11.0,10.602921,32.772664,13.494626,2.43,19.278037
184,E07000169,Selby,311.0,343.191348,90620.0,599.0,42.0,46.347385,5.0,5.517546,32.001766,15.449128,2.07,16.552637
66,E07000028,Carlisle,646.0,594.416533,108678.0,1039.0,36.0,33.125380,21.0,19.323138,18.402989,5.520897,3.33,12.882092
279,E08000034,Kirklees,1960.0,445.670290,439787.0,409.0,498.0,113.236635,112.0,25.466874,57.755232,45.249177,1.28,12.506054
133,E07000112,Folkestone and Hythe,712.0,630.110800,112996.0,357.0,126.0,111.508372,23.0,20.354703,57.524160,46.019328,1.25,11.504832
206,E07000198,Staffordshire Moorlands,364.0,369.787169,98435.0,576.0,53.0,53.842637,13.0,13.206685,32.508762,21.333875,1.52,11.174887
155,E07000134,North West Leicestershire,300.0,289.544546,103611.0,279.0,49.0,47.292276,3.0,2.895445,31.849900,21.233267,1.50,10.616633
257,E08000011,Knowsley,986.0,653.577442,150862.0,87.0,72.0,47.725736,21.0,13.920006,27.177155,16.571436,1.64,10.605719


In [28]:
ltlasAllSumDf[ltlasAllSumDf.areaName=='Warwick']

,areaCode,areaName,dailyLabConfirmedCases,rate,Population,Area,last30dCases,last30dRate,last7dCases,last7dRate
226,E07000222,Warwick,430.0,299.124192,143753.0,283.0,16.0,11.130203,1.0,0.695638


In [ ]:
ltlasDf.specimenDate.max()

In [ ]:
ltlasDf.groupby(['areaName'])['rate'].sum().sort_values(ascending=False).head(10)

In [ ]:
ltlasWorst10RateLast30D = ltlasDf[ltlasDf.specimenDate > ltlasDf.specimenDate.max()- timedelta(days=30)].groupby(['areaName'])['rate'].sum().sort_values(ascending=False).head(10).reset_index()

In [ ]:
ltlasWorst10RateLast30D.rate = np.round(ltlasWorst10RateLast30D.rate,1)

In [ ]:
ltlasSumDf = ltlasDf.groupby('areaCode')['dailyLabConfirmedCases'].sum().reset_index()
tmp = ltlasDf[ltlasDf.specimenDate > (ltlasDf.specimenDate.max() - timedelta(days=30))].groupby('areaCode')['dailyLabConfirmedCases'].sum()
ltlasSumDf['last30dCases'] = ltlasSumDf['areaCode'].map(tmp)
tmp = ltlasDf[(ltlasDf.specimenDate >= (ltlasDf.specimenDate.max() - timedelta(days=17))) & (ltlasDf.specimenDate <= (ltlasDf.specimenDate.max() - timedelta(days=4)))].groupby('areaCode')['dailyLabConfirmedCases'].sum()
tmp1 = ltlasDf[(ltlasDf.specimenDate >= (ltlasDf.specimenDate.max() - timedelta(days=31))) & (ltlasDf.specimenDate <= (ltlasDf.specimenDate.max() - timedelta(days=18)))].groupby('areaCode')['dailyLabConfirmedCases'].sum()
ltlasSumDf['rFirst14'] = ltlasSumDf['areaCode'].map(tmp)
ltlasSumDf['rSecond14'] = ltlasSumDf['areaCode'].map(tmp1)
ltlasSumDf['rBasic'] = np.round(ltlasSumDf['rFirst14'] /  ltlasSumDf['rSecond14'],2)
ltlasSumDf.dailyLabConfirmedCases = ltlasSumDf.dailyLabConfirmedCases.astype('int')
ltlasSumDf.last30dCases = ltlasSumDf.last30dCases.astype('int')
ltlasSumDf.rFirst14 = ltlasSumDf.rFirst14.astype('int')
ltlasSumDf.rSecond14= ltlasSumDf.rSecond14.astype('int')
ltlasSumDf['rBasic'] = ltlasSumDf['rBasic'].fillna(0)
ltlasSumDf['rBasic'] = ltlasSumDf['rBasic'].replace(np.inf, ltlasSumDf['rFirst14'])

In [ ]:
tmp = ltlasDf[(ltlasDf.specimenDate >= (ltlasDf.specimenDate.max() - timedelta(days=17))) & (ltlasDf.specimenDate <= (ltlasDf.specimenDate.max() - timedelta(days=4)))].groupby('areaCode')['rate'].sum()
tmp1 = ltlasDf[(ltlasDf.specimenDate >= (ltlasDf.specimenDate.max() - timedelta(days=31))) & (ltlasDf.specimenDate <= (ltlasDf.specimenDate.max() - timedelta(days=18)))].groupby('areaCode')['rate'].sum()
ltlasSumDf['rFirst14Rate'] = ltlasSumDf['areaCode'].map(tmp)
ltlasSumDf['rSecond14Rate'] = ltlasSumDf['areaCode'].map(tmp1)
ltlasSumDf['rBasicRate'] = np.round(ltlasSumDf['rFirst14Rate'] /  ltlasSumDf['rSecond14Rate'],2)


In [ ]:
ltlasSumDf

In [ ]:
ltlasSumDf.dailyLabConfirmedCases = ltlasSumDf.dailyLabConfirmedCases.astype('int')
ltlasSumDf.last30dCases = ltlasSumDf.last30dCases.astype('int')
ltlasSumDf.rFirst14 = ltlasSumDf.rFirst14.astype('int')
ltlasSumDf.rSecond14= ltlasSumDf.rSecond14.astype('int')
ltlasSumDf['rBasic'] = ltlasSumDf['rBasic'].fillna(0)
ltlasSumDf['rBasic'] = ltlasSumDf['rBasic'].replace(np.inf, ltlasSumDf['rFirst14'])

In [ ]:
tmp = lowerToUpperDf[['LTLA19CD','LTLA19NM']].drop_duplicates()
tmp = tmp.rename(columns={'LTLA19CD': 'areaCode','LTLA19NM': 'areaName'})
tmp = pd.merge(
    left=ltlasSumDf,
    right=tmp,
    how='left')
ltlasWorst10Df = tmp[tmp.rFirst14>=20][['areaName','rFirst14','rSecond14','rBasic']].sort_values(by='rBasic', ascending=False).head(10)
ltlastop10last30dDf = tmp[['areaName','last30dCases']].sort_values(by='last30dCases', ascending=False).head(10)

In [ ]:
print(ltlasDf.specimenDate.max())
print(ltlasDf.specimenDate.max() - timedelta(days=17), ltlasDf.specimenDate.max() - timedelta(days=4))
print(ltlasDf.specimenDate.max() - timedelta(days=31), ltlasDf.specimenDate.max() - timedelta(days=18))

In [ ]:
ltlasSumDf[ltlasSumDf.rSecond14>30].sort_values(by='rBasic', ascending=False).head(20)

In [ ]:
today : 22
first : 4-17
second : 21-3

In [ ]:
ltlasDf = ltlasDf[ltlasDf.specimenDate>='2020-03-01']

ltlasDf = ltlasDf.rename(
columns={
    "dailyLabConfirmedCases" : "dcLower",
    "totalLabConfirmedCases" : "tcLower",
    "upperRegionCode" : "urCode",
    "upperRegionName" : "urName",
    "dailyLabConfirmedCasesUpperRegion" : "dcUpper",
    "regionCode" : "rCode",
    "regionName" : "rName",
    "dailyLabConfirmedCasesRegion" : "dcRegion",
    "areaMovingAverage7" : "ma7Lower",
    "upperRegionMovingAverage7" : "ma7Upper",
    "regionMovingAverage7" : "ma7Region"
    }
)

In [ ]:
ltlasDf.to_json(path_or_buf="data/ltlas.json", orient="records", date_format='iso')

In [ ]:
ltlasDf[ltlasDf.areaName.str.len()==35]